In [40]:
import numpy as np
from wikidata.client import Client
from collections import defaultdict
import torch
client = Client()
%load_ext autoreload
%autoreload 2
from data import analysis, modify

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
data_dir = "./data/clean/wd50k/statements_switch"

In [37]:
train_data, num_train = analysis.read_data(data_dir + "/train.txt")
val_data, num_val = analysis.read_data(data_dir + "/valid.txt")
test_data, num_test = analysis.read_data(data_dir + "/test.txt")

num tuples:  199761
num tuples:  23913
num tuples:  46159


In [17]:
# "sub: [[rel_main, obj_main], [rel_qual_1, obj_qual_1], [rel_qual_2, obj_qual_2], ...]"
test_data["Q55294"]

[[['P1411', 'Q107258'], ['P805', 'Q903336'], ['P1686', 'Q898721']]]

In [27]:
# with open(data_dir + "/test_name.txt", "w") as wf:
#     for ary in data[:200]:
#         wf.write("||".join([str(client.get(ele).label) for ele in ary]) + "\n")

In [38]:
overlap, overlap_r = [], []
for ent in test_data:
    for edge in test_data[ent]:
        main_triple = [ent] + edge[0]
        main_triple_reverse = [edge[0][1], edge[0][0], ent]
        if analysis.test_triple_2(main_triple, train_data):
            overlap.append(main_triple)
        if analysis.test_triple_2(main_triple_reverse, train_data):
            overlap_r.append(main_triple_reverse)
print(len(overlap)/num_test)
print(len(overlap_r)/num_test)

0.001343183344526528
0.031369830368942135


In [35]:
# test whether test data leakage problem exists: one main triple exists in the data dict
overlap, overlap_r = [], []
for ent in test_data:
    for edge in test_data[ent]:
        main_triple = [ent] + edge[0]
        main_triple_reverse = [edge[0][1], edge[0][0], ent]
        if analysis.test_triple(main_triple, train_data):
            overlap.append(main_triple)
        if analysis.test_triple(main_triple_reverse, train_data):
            overlap_r.append(main_triple_reverse)
print(len(overlap)/num_test)
print(len(overlap_r)/num_test)

0.001343183344526528
0.031369830368942135


In [60]:
# test whether one qualifier is also a main triple
overlap = []
num_qual = 0
for ent in train_data:
    for edge in train_data[ent]:
        for qual_edge in edge[1:]:
            num_qual += 1
            qual_triple = [ent] + qual_edge
            qual_triple_reverse = [qual_edge[1], qual_edge[0], ent]
            if analysis.test_triple_2(qual_triple, train_data) \
                or analysis.test_triple_2(qual_triple_reverse, train_data):
                overlap.append(qual_triple)
print(len(overlap)/num_qual)

0.006211366500630138


In [24]:
# how many entities/relations only exists in qualifiers
entity_set_main = analysis.get_entity_set_main(train_data)
entity_set_qual = analysis.get_entity_set_qual(train_data)
print("number of entities in main: ", len(entity_set_main))
print("number of entities in qual: ", len(entity_set_qual))
print("number of entities in all: ", len(entity_set_main.union(entity_set_qual)))
print("number of entities only in qual: ", len(entity_set_qual - entity_set_main))

number of entities in main:  9890
number of entities in qual:  13035
number of entities in all:  17684
number of entities only in qual:  7794


In [25]:
# how many entities/relations only exists in qualifiers
relation_set_main = analysis.get_relation_set_main(train_data)
relation_set_qual = analysis.get_relation_set_qual(train_data)
print("number of relations in main: ", len(relation_set_main))
print("number of relations in qual: ", len(relation_set_qual))
print("number of relations in all: ", len(relation_set_main.union(relation_set_qual)))
print("number of relations only in qual: ", len(relation_set_qual - relation_set_main))

number of relations in main:  197
number of relations in qual:  132
number of relations in all:  271
number of relations only in qual:  74


## Generate new data

In [28]:
%%bash
data_dir="./data/clean/wd50k"
cp -r ${data_dir}/statements ${data_dir}/statements_switch

In [29]:
data_dir = "./data/clean/wd50k/statements_switch"
modify.aug_switch(data_dir + "/train.txt", data_dir + "/train_switch.txt")

In [31]:
%%bash
data_dir="./data/clean/wd50k/statements_switch"
mv ${data_dir}/train_switch.txt ${data_dir}/train.txt 

In [48]:
a=torch.randn(5)
print(a)
print(torch.argsort(a, descending=True))
torch.argsort(torch.argsort(a, descending=True))

tensor([-0.2562, -0.5812, -1.1616, -0.9717, -0.4676])
tensor([0, 4, 1, 3, 2])


tensor([0, 2, 4, 3, 1])